### Google Drive Mounting and File Paths Setup in Colab

In [1]:
"""
Mounting the Google drive
"""
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
root_path = "/content/drive/MyDrive/Colab Notebooks/training/"
train_path = root_path + "train/"
validate_path = root_path + "validate/"
test_path = root_path + "test/"

### Image Processing Functions for Data Augmentation

In [ ]:
import numpy as np
import cv2
import os
from google.colab.patches import cv2_imshow

def translate_image(image, dx, dy, path):
    translation_matrix = np.float32([[1, 0, dx], [0, 1, dy]])
    translated_image = cv2.warpAffine(image, translation_matrix, (image.shape[1], image.shape[0]))
    translated_image = face_box(translated_image)
    img_name = f'{image_name}_translated_dx{dx}_dy{dy}.jpeg'
    if not os.path.isfile(os.path.join(path, img_name)) and translated_image is not None:
      save_path = os.path.join(path, img_name)
      cv2.imwrite(save_path, translated_image)
      write_to_file(img_name, path)

def rotate_image(image, angle, path):
    rows, cols = image.shape[:2]
    M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
    rotated_image = cv2.warpAffine(image, M, (cols,rows))
    rotated_image = face_box(rotated_image)
    img_name = f'{image_name}_rotated_angle{angle}.jpeg'
    if not os.path.isfile(os.path.join(path, img_name)) and rotated_image is not None:
      save_path = os.path.join(path, img_name)
      cv2.imwrite(save_path, rotated_image)
      write_to_file(img_name, path)

def scale_image(image, path, scale_factor):
    img_shrinked = cv2.resize(image, (200, 200), interpolation = cv2.INTER_AREA)
    img_shrinked = face_box(img_shrinked)
    img_name = f'{image_name}_shrinked_img.jpeg'
    if not os.path.isfile(os.path.join(path, img_name)) and img_shrinked is not None:
      save_path = os.path.join(path, img_name)
      cv2.imwrite(save_path, img_shrinked)
      write_to_file(img_name, path)
    img_enlarged = cv2.resize(image, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)
    img_enlarged = face_box(img_enlarged)
    enlargedimg_name = f'{image_name}_enlarged_img.jpeg'
    if not os.path.isfile(os.path.join(path, enlargedimg_name)) and img_enlarged is not None:
      save_path = os.path.join(path, enlargedimg_name)
      cv2.imwrite(save_path, img_enlarged)
      write_to_file(enlargedimg_name, path)


def adjust_brightness(image, value, path):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv[:, :, 2] += value
    hsv[:, :, 2] = np.clip(hsv[:, :, 2], 0, 255)
    adjusted_image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    adjusted_image = face_box(adjusted_image)
    img_name = f'{image_name}_brightness_value{value}.jpeg'
    if not os.path.isfile(os.path.join(path, img_name)) and adjusted_image is not None:
      save_path = os.path.join(path, img_name)
      cv2.imwrite(save_path, adjusted_image)
      write_to_file(img_name, path)

def adjust_contrast(image, value, path):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    lab[:, :, 0] = np.clip(lab[:, :, 0] * value, 0, 255)
    adjusted_image = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    adjusted_image = face_box(adjusted_image)
    img_name = f'{image_name}_contrast_value{value}.jpeg'
    if not os.path.isfile(os.path.join(path, img_name)) and adjusted_image is not None:
      save_path = os.path.join(path, img_name)
      cv2.imwrite(save_path, adjusted_image)
      write_to_file(img_name, path)

def adjust_saturation(image, value, path):
    hls = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
    hls[:, :, 2] += value
    hls[:, :, 2] = np.clip(hls[:, :, 2], 0, 255)
    adjusted_image = cv2.cvtColor(hls, cv2.COLOR_HLS2BGR)
    adjusted_image = face_box(adjusted_image)
    img_name = f'{image_name}_saturation_value{value}.jpeg'
    if not os.path.isfile(os.path.join(path, img_name)) and adjusted_image is not None:
      save_path = os.path.join(path, img_name)
      cv2.imwrite(save_path, adjusted_image)
      write_to_file(img_name, path)

def gaussian_blur(image, path):
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    smooth = cv2.GaussianBlur(gray, (125,125), 0)
    division = cv2.divide(gray, smooth, scale=255)
    adjusted_image = face_box(division)

    img_name = f'{image_name}_gaussian_blur.jpeg'
    if not os.path.isfile(os.path.join(path, img_name)) and adjusted_image is not None:
      save_path = os.path.join(path, img_name)
      cv2.imwrite(save_path, adjusted_image)
      write_to_file(img_name, path)

def write_to_file(image_name, path):
    entry = f'\n{image_name} {label}'
    file_path = os.path.join(path, "file_mapping.txt")
    with open(file_path, 'r') as file:
        existing_entries = set(line.strip() for line in file)
    if entry not in existing_entries:
        with open(file_path, 'a') as file:
            file.write(entry)

def face_box(image):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces_detected = face_cascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5)
    for (x, y, w, h) in faces_detected:
      image_face = image[y:y+h, x:x+w]
      resized_image_face = cv2.resize(image_face, (128, 128))
      return resized_image_face


### Data Augmentation and Image Generation for Validation Set

#### Image Augmentation Pipeline for Raw Data Enhancement

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
f = open(validate_path + "file_mapping.txt", "r")
next(f)
for line in f:
  if not line.strip():  # Skip blank lines
        continue

  image_name, label = line.split(" ")
  if image_name == "Modified_images":
    print(image_name)
    break
  print(image_name, label)
  image = cv2.imread(validate_path + image_name)

  #1. translation
  for dx, dy in [(25, 20), (-25, -20)]:
      translate_image(image, dx, dy, validate_path)

  #2. flipping
  flipped_img = cv2.flip(image,1)
  flipped_img = face_box(flipped_img)
  flipimg_name = f'{image_name}_flipped.jpeg'
  if not os.path.isfile(os.path.join(validate_path, flipimg_name)) and flipped_img is not None:
    save_path = os.path.join(validate_path, flipimg_name)
    cv2.imwrite(save_path, flipped_img)
    write_to_file(flipimg_name, validate_path)

  #3. rotation
  for angle in [7, -7, 10, -10]:
      rotate_image(image, angle, validate_path)

  #4. background-lighting
  brightness = 3
  contrast = 1.6
  bc_image = cv2.addWeighted(image, contrast, np.zeros(image.shape, image.dtype), 0, brightness)
  bc_image = face_box(bc_image)
  bkgAdimg_name = f'{image_name}_backgroundAdj.jpeg'
  if not os.path.isfile(os.path.join(validate_path, bkgAdimg_name)) and bc_image is not None:
    save_path = os.path.join(validate_path, bkgAdimg_name)
    cv2.imwrite(save_path, bc_image)
    write_to_file(bkgAdimg_name, validate_path)

  #5. scaling
  scale_image(image, validate_path, 1.5)

  #6. brightness
  for value in [18, 20, 22]:
    adjust_brightness(image, value, validate_path)

  #.7 contrast
  for value in [1.3, 1.5, 1.7]:
    adjust_contrast(image, value, validate_path)

  #8. Adjust saturation
  for value in [50, 60, 70]:
    adjust_saturation(image, value, validate_path)

  #9. sheared-y
  rows, cols = image.shape[:2]
  M = np.float32([[1,   0, 0], [0.5, 1, 0], [0,   0, 1]])
  sheared_yimg = cv2.warpPerspective(image, M, (int(cols*1.5), int(rows*1.5)))
  sheared_yimg = face_box(sheared_yimg)

  shearyimg_name = f'{image_name}_sheared_yimg.jpeg'
  if not os.path.isfile(os.path.join(validate_path, shearyimg_name)) and sheared_yimg is not None:
    save_path = os.path.join(validate_path, shearyimg_name)
    cv2.imwrite(save_path, sheared_yimg)
    write_to_file(shearyimg_name, validate_path)

  #10. sheared-x
  Ma = np.float32([[1, 0.5, 0], [0, 1, 0], [0, 0, 1]])
  sheared_ximg = cv2.warpPerspective(image, Ma, (int(cols*1.5), int(rows*1.5)))
  sheared_ximg = face_box(sheared_ximg)
  print(np.shape(np.array(sheared_ximg)))
  shearximg_name = f'{image_name}_sheared_ximg.jpeg'
  if not os.path.isfile(os.path.join(validate_path, shearximg_name)) and sheared_ximg is not None:
    save_path = os.path.join(validate_path, shearximg_name)
    cv2.imwrite(save_path, sheared_ximg)
    write_to_file(shearximg_name, validate_path)

  #11. Gaussian Blur
  gaussian_blur(image, validate_path)

  print("Generated all images")

0206_1.jpeg sampagaonrahul

(128, 128, 3)
Generated all images
0206_2.jpeg sivarajusairevanth

(128, 128, 3)
Generated all images
0206_3.jpeg pereiranerissagodfrey

(128, 128, 3)
Generated all images
0206_4.jpeg wukaiyue



KeyboardInterrupt: 

#### Image Augmentation Pipeline for Rotated Data Enhancement

In [ ]:
def modified_img(image_name, keyword):
  return keyword in image_name

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

keywords = ["flipped","backgroundAdj", "shrinked_img", "enlarged_img", "brightness_value", "contrast_value", "saturation_value", "sheared_yimg",\
            "sheared_ximg", "gaussian_blur"]
f = open(validate_path + "file_mapping.txt", "r")
next(f)

for line in f:
  if not line.strip():  # Skip blank lines
        continue

  image_name, label = line.split(" ")

  if not modified_img(image_name, "rotated_angle"):
    continue

  if any([modified_img(image_name, keyword) for keyword in keywords]):
    continue

  print(image_name, label)
  image = cv2.imread(validate_path + image_name)


  #1. flipping
  flipped_img = cv2.flip(image,1)
  flipped_img = face_box(flipped_img)
  flipimg_name = f'{image_name}_flipped.jpeg'
  if not os.path.isfile(os.path.join(validate_path, flipimg_name)) and flipped_img is not None:
    save_path = os.path.join(validate_path, flipimg_name)
    cv2.imwrite(save_path, flipped_img)
    write_to_file(flipimg_name, validate_path)

  #2. background-lighting
  brightness = 3
  contrast = 1.6
  bc_image = cv2.addWeighted(image, contrast, np.zeros(image.shape, image.dtype), 0, brightness)
  bc_image = face_box(bc_image)
  bkgAdimg_name = f'{image_name}_backgroundAdj.jpeg'
  if not os.path.isfile(os.path.join(validate_path, bkgAdimg_name)) and bc_image is not None:
    save_path = os.path.join(validate_path, bkgAdimg_name)
    cv2.imwrite(save_path, bc_image)
    write_to_file(bkgAdimg_name, validate_path)


  #4. brightness
  for value in [18, 20, 22]:
    adjust_brightness(image, value, validate_path)

  #5. contrast
  for value in [1.3, 1.5, 1.7]:
    adjust_contrast(image, value, validate_path)

  #6. Adjust saturation
  for value in [50, 60, 70]:
    adjust_saturation(image, value, validate_path)

  #7. sheared-y
  rows, cols = image.shape[:2]
  M = np.float32([[1,   0, 0], [0.5, 1, 0], [0,   0, 1]])
  sheared_yimg = cv2.warpPerspective(image, M, (int(cols*1.5), int(rows*1.5)))
  sheared_yimg = face_box(sheared_yimg)

  shearyimg_name = f'{image_name}_sheared_yimg.jpeg'
  if not os.path.isfile(os.path.join(validate_path, shearyimg_name)) and sheared_yimg is not None:
    save_path = os.path.join(validate_path, shearyimg_name)
    cv2.imwrite(save_path, sheared_yimg)
    write_to_file(shearyimg_name, validate_path)

  #8. sheared-x
  Ma = np.float32([[1, 0.5, 0], [0, 1, 0], [0, 0, 1]])
  sheared_ximg = cv2.warpPerspective(image, Ma, (int(cols*1.5), int(rows*1.5)))
  sheared_ximg = face_box(sheared_ximg)
  print(np.shape(np.array(sheared_ximg)))
  shearximg_name = f'{image_name}_sheared_ximg.jpeg'
  if not os.path.isfile(os.path.join(validate_path, shearximg_name)) and sheared_ximg is not None:
    save_path = os.path.join(validate_path, shearximg_name)
    cv2.imwrite(save_path, sheared_ximg)
    write_to_file(shearximg_name, validate_path)

  #11. Gaussian Blur
  gaussian_blur(image, validate_path)

  print("Generated all images")

0206_1.jpeg_rotated_angle7.jpeg sampagaonrahul

(256, 256, 3)
Generated all images
0206_1.jpeg_rotated_angle-7.jpeg sampagaonrahul

(256, 256, 3)
Generated all images
0206_1.jpeg_rotated_angle10.jpeg sampagaonrahul

(256, 256, 3)
Generated all images
0206_1.jpeg_rotated_angle-10.jpeg sampagaonrahul

(256, 256, 3)
Generated all images
0206_1.jpeg_rotated_angle7.jpeg sampagaonrahul

(256, 256, 3)
Generated all images
0206_1.jpeg_rotated_angle-7.jpeg sampagaonrahul

(256, 256, 3)
Generated all images
0206_1.jpeg_rotated_angle10.jpeg sampagaonrahul

(256, 256, 3)
Generated all images
0206_1.jpeg_rotated_angle-10.jpeg sampagaonrahul

(256, 256, 3)
Generated all images
0206_1.jpeg_rotated_angle7.jpeg sampagaonrahul

(256, 256, 3)
Generated all images
0206_1.jpeg_rotated_angle-7.jpeg sampagaonrahul

(256, 256, 3)
Generated all images
0206_1.jpeg_rotated_angle10.jpeg sampagaonrahul

(256, 256, 3)
Generated all images
0206_1.jpeg_rotated_angle-10.jpeg sampagaonrahul

(256, 256, 3)
Generated all

## Data Augmentation and Image Generation for Train Set

#### Image Augmentation Pipeline for Raw Data Enhancement

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
f = open(train_path + "file_mapping.txt", "r")
next(f)
for line in f:
  if not line.strip():  # Skip blank lines
        continue

  image_name, label = line.split(" ")
  if image_name == "Modified_image":
    print(image_name)
    break
  print(image_name, label)
  image = cv2.imread(train_path + image_name)

  # if not image: continue
  #1. translation
  for dx, dy in [(25, 20), (-25, -20)]:
      translate_image(image, dx, dy, train_path)

  #2. flipping
  flipped_img = cv2.flip(image,1)
  flipped_img = face_box(flipped_img)
  flipimg_name = f'{image_name}_flipped.jpeg'
  if not os.path.isfile(os.path.join(train_path, flipimg_name)) and flipped_img is not None:
    save_path = os.path.join(train_path, flipimg_name)
    cv2.imwrite(save_path, flipped_img)
    write_to_file(flipimg_name, train_path)

  #3. rotation
  for angle in [7, -7, 10, -10]:
      rotate_image(image, angle, train_path)

  #4. background-lighting
  brightness = 3
  contrast = 1.6
  bc_image = cv2.addWeighted(image, contrast, np.zeros(image.shape, image.dtype), 0, brightness)
  bc_image = face_box(bc_image)
  bkgAdimg_name = f'{image_name}_backgroundAdj.jpeg'
  if not os.path.isfile(os.path.join(train_path, bkgAdimg_name)) and bc_image is not None:
    save_path = os.path.join(train_path, bkgAdimg_name)
    cv2.imwrite(save_path, bc_image)
    write_to_file(bkgAdimg_name, train_path)

  #5. scaling
  scale_image(image, train_path, 1.5)

  #6. brightness
  for value in [18, 20, 22]:
    adjust_brightness(image, value, train_path)

  #.7 contrast
  for value in [1.3, 1.5, 1.7]:
    adjust_contrast(image, value, train_path)

  #8. Adjust saturation
  for value in [50, 60, 70]:
    adjust_saturation(image, value, train_path)

  #9. sheared-y
  rows, cols = image.shape[:2]
  M = np.float32([[1,   0, 0], [0.5, 1, 0], [0,   0, 1]])
  sheared_yimg = cv2.warpPerspective(image, M, (int(cols*1.5), int(rows*1.5)))
  sheared_yimg = face_box(sheared_yimg)

  shearyimg_name = f'{image_name}_sheared_yimg.jpeg'
  if not os.path.isfile(os.path.join(train_path, shearyimg_name)) and sheared_yimg is not None:
    save_path = os.path.join(train_path, shearyimg_name)
    cv2.imwrite(save_path, sheared_yimg)
    write_to_file(shearyimg_name, train_path)

  #10. sheared-x
  Ma = np.float32([[1, 0.5, 0], [0, 1, 0], [0, 0, 1]])
  sheared_ximg = cv2.warpPerspective(image, Ma, (int(cols*1.5), int(rows*1.5)))
  sheared_ximg = face_box(sheared_ximg)
  print(np.shape(np.array(sheared_ximg)))
  shearximg_name = f'{image_name}_sheared_ximg.jpeg'
  if not os.path.isfile(os.path.join(train_path, shearximg_name)) and sheared_ximg is not None:
    save_path = os.path.join(train_path, shearximg_name)
    cv2.imwrite(save_path, sheared_ximg)
    write_to_file(shearximg_name, train_path)

  #11. Gaussian Blur
  gaussian_blur(image, train_path)

  print("Generated all images")

0123_1.jpeg wukaiyue

(128, 128, 3)
Generated all images
0123_2.jpeg vanderlindenilona

(128, 128, 3)
Generated all images
0123_3.jpeg zhangyuanzhen

(128, 128, 3)
Generated all images
0123_4.jpeg liuhongji

(128, 128, 3)
Generated all images
0123_5.jpeg lozanoroberto

(128, 128, 3)
Generated all images
0123_6.jpeg yashasvi

(128, 128, 3)
Generated all images
0123_7.jpeg huangjiaoyan

(128, 128, 3)
Generated all images
0123_8.jpeg manglaniroshanlakhi

(128, 128, 3)
Generated all images
0123_9.jpeg somaniachal

(128, 128, 3)
Generated all images
0123_10.jpeg zotaharsh

(128, 128, 3)
Generated all images
0123_11.jpeg pereiranerissagodfrey

(128, 128, 3)
Generated all images
0123_12.jpeg zuluagagonzalezisabel

(128, 128, 3)
Generated all images
0123_13.jpeg lishumeng

(128, 128, 3)
Generated all images
0123_14.jpeg ravijayanthidhanasekar

(128, 128, 3)
Generated all images
0123_15.jpeg oraisisaac

(128, 128, 3)
Generated all images
0123_16.jpeg kodipunzulanandini

(128, 128, 3)
Generated 

#### Image Augmentation Pipeline for Rotated Data Enhancement

In [ ]:
def modified_img(image_name, keyword):
  return keyword in image_name

import cv2
import numpy as np
from google.colab.patches import cv2_imshow

keywords = ["flipped","backgroundAdj", "shrinked_img", "enlarged_img", "brightness_value", "contrast_value", "saturation_value", "sheared_yimg",\
            "sheared_ximg", "gaussian_blur"]
f = open(train_path + "file_mapping.txt", "r")
next(f)

for line in f:
  if not line.strip():  # Skip blank lines
        continue

  image_name, label = line.split(" ")

  if not modified_img(image_name, "rotated_angle"):
    continue

  if any([modified_img(image_name, keyword) for keyword in keywords]):
    continue

  print(image_name, label)
  image = cv2.imread(train_path + image_name)


  #1. flipping
  flipped_img = cv2.flip(image,1)
  flipped_img = face_box(flipped_img)
  flipimg_name = f'{image_name}_flipped.jpeg'
  if not os.path.isfile(os.path.join(train_path, flipimg_name)) and flipped_img is not None:
    save_path = os.path.join(train_path, flipimg_name)
    cv2.imwrite(save_path, flipped_img)
    write_to_file(flipimg_name, train_path)

  #2. background-lighting
  brightness = 3
  contrast = 1.6
  bc_image = cv2.addWeighted(image, contrast, np.zeros(image.shape, image.dtype), 0, brightness)
  bc_image = face_box(bc_image)
  bkgAdimg_name = f'{image_name}_backgroundAdj.jpeg'
  if not os.path.isfile(os.path.join(train_path, bkgAdimg_name)) and bc_image is not None:
    save_path = os.path.join(train_path, bkgAdimg_name)
    cv2.imwrite(save_path, bc_image)
    write_to_file(bkgAdimg_name, train_path)


  #4. brightness
  for value in [18, 20, 22]:
    adjust_brightness(image, value, train_path)

  #5. contrast
  for value in [1.3, 1.5, 1.7]:
    adjust_contrast(image, value, train_path)

  #6. Adjust saturation
  for value in [50, 60, 70]:
    adjust_saturation(image, value, train_path)

  #7. sheared-y
  rows, cols = image.shape[:2]
  M = np.float32([[1,   0, 0], [0.5, 1, 0], [0,   0, 1]])
  sheared_yimg = cv2.warpPerspective(image, M, (int(cols*1.5), int(rows*1.5)))
  sheared_yimg = face_box(sheared_yimg)

  shearyimg_name = f'{image_name}_sheared_yimg.jpeg'
  if not os.path.isfile(os.path.join(train_path, shearyimg_name)) and sheared_yimg is not None:
    save_path = os.path.join(train_path, shearyimg_name)
    cv2.imwrite(save_path, sheared_yimg)
    write_to_file(shearyimg_name, train_path)

  #8. sheared-x
  Ma = np.float32([[1, 0.5, 0], [0, 1, 0], [0, 0, 1]])
  sheared_ximg = cv2.warpPerspective(image, Ma, (int(cols*1.5), int(rows*1.5)))
  sheared_ximg = face_box(sheared_ximg)
  print(np.shape(np.array(sheared_ximg)))
  shearximg_name = f'{image_name}_sheared_ximg.jpeg'
  if not os.path.isfile(os.path.join(train_path, shearximg_name)) and sheared_ximg is not None:
    save_path = os.path.join(train_path, shearximg_name)
    cv2.imwrite(save_path, sheared_ximg)
    write_to_file(shearximg_name, train_path)

  #11. Gaussian Blur
  gaussian_blur(image, train_path)

  print("Generated all images")

0123_1.jpeg_rotated_angle7.jpeg wukaiyue

()
Generated all images
0123_1.jpeg_rotated_angle-7.jpeg wukaiyue

(128, 128, 3)
Generated all images
0123_1.jpeg_rotated_angle10.jpeg wukaiyue

()
Generated all images
0123_1.jpeg_rotated_angle-10.jpeg wukaiyue

(128, 128, 3)
Generated all images
0123_2.jpeg_rotated_angle7.jpeg vanderlindenilona

(128, 128, 3)
Generated all images
0123_2.jpeg_rotated_angle-7.jpeg vanderlindenilona

(128, 128, 3)
Generated all images
0123_2.jpeg_rotated_angle10.jpeg vanderlindenilona

(128, 128, 3)
Generated all images
0123_2.jpeg_rotated_angle-10.jpeg vanderlindenilona

(128, 128, 3)
Generated all images
0123_3.jpeg_rotated_angle7.jpeg zhangyuanzhen

(128, 128, 3)
Generated all images
0123_3.jpeg_rotated_angle-7.jpeg zhangyuanzhen

(128, 128, 3)
Generated all images
0123_3.jpeg_rotated_angle10.jpeg zhangyuanzhen

(128, 128, 3)
Generated all images
0123_3.jpeg_rotated_angle-10.jpeg zhangyuanzhen

(128, 128, 3)
Generated all images
0123_4.jpeg_rotated_angle7.jpe

## Dimension Reduction

### Loading labels

In [ ]:
f = open(train_path + "file_mapping.txt", "r")
next(f)
labels_dict = {}
for line in f:
  if not line.strip():  # Skip blank lines
        continue

  image_name, label = line.split(" ")
  if image_name == "Modified_image": continue
  labels_dict[image_name] = label.strip()

### Loading training images with label

In [ ]:
import os
import cv2
import numpy as np

def load_images(directory_path, label):
    images = []
    labels = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".jpeg"):
            img_path = os.path.join(directory_path, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (128,128))
            images.append(img.flatten())
            labels.append(labels_dict[filename])
    return images, labels

data, labels = load_images(train_path, labels_dict)
data = np.array(data)
labels = np.array(labels)

np.save(train_path + 'data.npy', data)
np.save(train_path + 'labels.npy', labels)

### Loading test images with labels

In [ ]:
import os
import cv2
import numpy as np

f = open(test_path + "file_mapping.txt", "r")
next(f)
labels_dict_test = {}
for line in f:
  if not line.strip():  # Skip blank lines
        continue

  image_name, label = line.split(" ")
  if image_name == "Modified_image": continue
  labels_dict_test[image_name] = label.strip()

def load_images(directory_path, label):
    images = []
    labels = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".jpeg"):
            img_path = os.path.join(directory_path, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (128,128))
            images.append(img.flatten())
            labels.append(labels_dict_test[filename])
    return images, labels

data_test, labels_test = load_images(test_path, labels_dict_test)
data_test = np.array(data_test)
labels_test = np.array(labels_test)

np.save(test_path + 'data.npy', data_test)
np.save(test_path + 'labels.npy', labels_test)

### Loading validation images with labels

In [ ]:
import os
import cv2
import numpy as np

f = open(validate_path + "file_mapping.txt", "r")
next(f)
labels_dict_validate = {}
for line in f:
  if not line.strip():  # Skip blank lines
        continue

  image_name, label = line.split(" ")
  if image_name == "Modified_image": continue
  labels_dict_validate[image_name] = label.strip()

def load_images(directory_path, label):
    images = []
    labels = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".jpeg"):
            img_path = os.path.join(directory_path, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (128,128))
            images.append(img.flatten())
            labels.append(labels_dict_validate[filename])
    return images, labels

data_validate, labels_validate = load_images(validate_path, labels_dict_validate)
data_validate = np.array(data_validate)
labels_validate = np.array(labels_validate)

np.save(validate_path + 'data.npy', data_validate)
np.save(validate_path + 'labels.npy', labels_validate)

KeyError: '0206_4.jpeg_rotated_angle-10.jpeg'

### Applying LDA

#### Importing data1

In [ ]:
import numpy as np

# loading train data
with open(train_path + 'data.npy', 'rb') as f:
    trainX = np.load(f)

with open(train_path + 'labels.npy', 'rb') as f:
    trainY = np.load(f)

#load validaition data
with open(validate_path + 'data.npy', 'rb') as f:
    testX = np.load(f)

with open(validate_path + 'labels.npy', 'rb') as f:
    testY = np.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/training/validate/data.npy'

#### Importing data 2

In [3]:
import numpy as np

with open(root_path + 'train.npy', 'rb') as f:
    trainX2 = np.load(f)
    trainY2 = np.load(f)

with open(root_path + 'test.npy', 'rb') as f:
    testX2 = np.load(f)
    testY2 = np.load(f)

In [4]:
print(trainX2.shape)
print(trainY2.shape)

(5834, 40000)
(5834,)


In [ ]:
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.datasets import fetch_lfw_people
import matplotlib.pyplot as plt


# Initialize LDA
n_components = 10  # You can adjust this as needed
lda = LinearDiscriminantAnalysis(n_components=n_components)

# Fit LDA to the training data
X_lda = lda.fit_transform(trainX2, trainY2)

# Plot explained variance ratio
plt.bar(range(n_components), lda.explained_variance_ratio_)
plt.xlabel('Number of Components')
plt.ylabel('Explained Variance Ratio')
plt.title('Explained Variance Ratio for LDA')
plt.show()

#### Applying PCA

In [5]:
from sklearn.decomposition import PCA
from skimage.exposure import rescale_intensity
from imutils import build_montages

import time

print("[INFO] creating eigenfaces...")
pca = PCA(
	svd_solver="randomized",
	n_components=100,
	whiten=True)
start = time.time()
trainX_PCA = pca.fit_transform(trainX2)
end = time.time()

[INFO] creating eigenfaces...


In [6]:
testX_PCA = pca.transform(testX2)

In [7]:
print(trainX2.shape)
print(trainX_PCA.shape)
print(testX_PCA.shape)

(5834, 40000)
(5834, 100)
(1251, 100)


#### Debugging LDA

##### Batch processing for LDA

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Assuming trainX and trainY are your input data and labels
# Convert labels to numerical values using LabelEncoder

label_encoder = LabelEncoder()
trainY_encoded = label_encoder.fit_transform(trainY[:17000])

# Create an LDA model
lda = LinearDiscriminantAnalysis()
# Initialize an empty list to store results
lda_results = []

# Process data in batches
batch_size = 1000
for i in range(0, len(trainX[:17000]), batch_size):
    batch_X = trainX[i:i + batch_size]
    batch_Y = trainY_encoded[i:i + batch_size]
    # batch_Y = trainY2[i:i + batch_size]

    # Fit LDA on the batch
    lda_result = lda.fit_transform(batch_X, batch_Y)
    print(batch_X.shape)
    # Append the results to the list
    lda_results.append(lda_result)

# Concatenate the results along axis 0 after the loop
lda_results = np.concatenate(lda_results, axis=0)

lda_results.shape

(1000, 49152)
(1000, 49152)
(1000, 49152)
(1000, 49152)
(1000, 49152)
(1000, 49152)
(1000, 49152)
(1000, 49152)
(1000, 49152)
(1000, 49152)
(1000, 49152)
(1000, 49152)
(1000, 49152)
(1000, 49152)
(1000, 49152)
(1000, 49152)
(1000, 49152)


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 27 and the array at index 1 has size 22

##### LDA on PCA

In [8]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib.pyplot as plt

lda = LinearDiscriminantAnalysis()

# Fit LDA to the training data
trainX_LDA = lda.fit_transform(trainX_PCA, trainY2)

In [9]:
testX_LDA = lda.transform(testX_PCA)

In [11]:
print("Original Dimensions:", trainX2.shape)
print("Reduced Dimensions PCA:", trainX_PCA.shape)
print("Reduced Dimensions LDA on PCA:", trainX_LDA.shape)
print("labels dim:", trainY2.shape)
print("testX:",testX2.shape)

Original Dimensions: (5834, 40000)
Reduced Dimensions PCA: (5834, 100)
Reduced Dimensions LDA on PCA: (5834, 32)
labels dim: (5834,)
testX: (1251, 40000)


## Applying models

### KNN

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()

# Fit and transform the training data
trainX_scaled = scaler.fit_transform(trainX_LDA)

# Transform the test data using the same scaler
testX_scaled = scaler.transform(testX_LDA)


# Define the parameter grid
param_grid = {'n_neighbors': [3, 5, 7, 9],
              'weights': ['uniform', 'distance'],
              'p': [1, 2]}

# Create the KNN model
knn_model = KNeighborsClassifier()

# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(knn_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(trainX_LDA, trainY2)

# Get the best parameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Train the KNN model with the best hyperparameters
best_knn_model = KNeighborsClassifier(**best_params)
best_knn_model.fit(trainX_LDA, trainY2)

# Make predictions and calculate accuracy
best_knn_predictions = best_knn_model.predict(testX_LDA)
best_knn_accuracy = accuracy_score(testY2, best_knn_predictions)
print("Best KNN Accuracy:", best_knn_accuracy)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best Hyperparameters: {'n_neighbors': 9, 'p': 2, 'weights': 'distance'}
Best KNN Accuracy: 0.8113509192645884


### SVM RBF

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler object
scaler = StandardScaler()

# Fit and transform the training data
trainX_scaled = scaler.fit_transform(trainX_LDA)

# Transform the test data using the same scaler
testX_scaled = scaler.transform(testX_LDA)

# Define the parameter grid for SVM with RBF kernel
param_grid_svm = {'C': [0.1, 1, 10],
                  'gamma': [0.01, 0.1, 1],
                  'kernel': ['rbf']}

# Create the SVM model with RBF kernel
svm_model = SVC()

# Use GridSearchCV to find the best hyperparameters
grid_search_svm = GridSearchCV(svm_model, param_grid_svm, cv=5, scoring='accuracy')
grid_search_svm.fit(trainX_LDA, trainY2)

# Get the best parameters for SVM
best_params_svm = grid_search_svm.best_params_
print("Best Hyperparameters for SVM:", best_params_svm)

# Train the SVM model with the best hyperparameters
best_svm_model = SVC(**best_params_svm)
best_svm_model.fit(trainX_LDA, trainY2)

# Make predictions and calculate accuracy for SVM
best_svm_predictions = best_svm_model.predict(testX_LDA)
best_svm_accuracy = accuracy_score(testY2, best_svm_predictions)
print("Best SVM Accuracy:", best_svm_accuracy)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best Hyperparameters for SVM: {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
Best SVM Accuracy: 0.8265387689848122


In [14]:
import pickle

# Save the trained SVM model to a Pickle file
with open(root_path + 'svm_model.pkl', 'wb') as file:
    pickle.dump(best_svm_model, file)
